In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from matplotlib import colors
 
pd.options.mode.chained_assignment = None  
import seaborn as sns


In [ ]:
sorted_ids = pd.read_csv('sorted_ids_24_JAN_2024.csv')
sorted_ids = sorted_ids[['participant_id', 'Days Of Data',  'period_of collection','min_date','max_date']]
sorted_ids = sorted_ids.rename(columns={'min_date':'initial_date', 'max_date':'final_date', 'Days Of Data':'days_with_metrics'})
sorted_ids

In [ ]:
patient = pd.read_parquet('clinical_data/Patients_lookup.parquet', engine='pyarrow')

patient['participant_id'] = patient['id'].apply(lambda x:x[:5])
patient = patient[patient['participant_id'].isin(sorted_ids['participant_id'].unique())][['participant_id', 'birth_date', 'gender']]
patient = patient.rename(columns={'birth_date':'birth_year'})
patient

In [ ]:
demographics = pd.read_excel('clinical_data/redcap_demographics_diagnosis_nov16.xlsx')
demographics = demographics[['participant_id','Year of birth PwD', 'Gender PwD','Diagnosis', 'alzheimer_diagnosis']]
demographics.head(3)

In [ ]:
patient = patient.merge(demographics,on='participant_id',how='left')
patient 

In [ ]:
mmse = pd.read_excel("clinical_data/redcap_mmse_baseline_values_nov30.xlsx")
mmse = mmse[mmse['closest_to_initial_day']==True]
mmse = mmse.rename(columns={'days_after_min_date':'date_difference_with_baseline_mmse',
                           'date_test_mmse':'date_baseline_mmse', 'tot_score_mmse':'baseline_mmse'})
mmse

In [ ]:
npi = pd.read_csv('clinical_data/npi_values_30_NOV_23_for_lme.csv')
npi = npi[npi['closest_to_initial_day']==True]
npi = npi.rename(columns={'time_in_study':'date_difference_baseline_npi','amended datetime':'date_baseline_npi',
                         'npi_total':'baseline_npi_total', 'npi_sleep':'baseline_npi_sleep'})
npi = npi[['participant_id','date_baseline_npi', 'date_difference_baseline_npi','min_date', 'baseline_npi_total',
           'baseline_npi_sleep']]
npi

In [ ]:
npi2 = pd.read_csv('clinical_data/npi_values_30_NOV_23_for_lme.csv')
npi2 = npi2[npi2['closest_to_final_day']==True]
npi2 = npi2.rename(columns={'time_in_study':'date_difference_final_npi','amended datetime':'date_final_npi',
                         'npi_total':'final_npi_total', 'npi_sleep':'final_npi_sleep'})
npi2 = npi2[['participant_id','date_final_npi', 'date_difference_final_npi','min_date', 'final_npi_total',
           'final_npi_sleep']]
npi2

In [ ]:
patient = patient.merge(mmse[['participant_id','date_baseline_mmse', 'baseline_mmse',]],on='participant_id',how='left')
patient = patient.merge(npi[['participant_id', 'date_baseline_npi','baseline_npi_total', 
                             'baseline_npi_sleep']],on='participant_id',how='left')
patient = patient.merge(npi2[['participant_id', 'date_final_npi','final_npi_total', 
                             'final_npi_sleep']],on='participant_id',how='left')
patient

In [ ]:
final = sorted_ids.merge(patient,on='participant_id')
final

In [ ]:
difference_function = lambda row: pd.to_datetime(row['initial_date'],dayfirst=True).year - row['Year of birth PwD'] if pd.notnull(row['Year of birth PwD']) else pd.to_datetime(row['initial_date'],dayfirst=True).year - float(row['birth_year'])
final['age_ate_start'] = final.apply(difference_function, axis=1)
final['gender'] = final['gender'].replace('m', 'Man')
final['gender'] = final['gender'].replace('f', 'Woman')
final['Gender PwD'] = final['Gender PwD'].replace('Male', 'Man')
final['Gender PwD'] = final['Gender PwD'].replace('Female', 'Woman')
final

In [ ]:
final.columns

In [ ]:
final.to_excel('table_demographics_24_JAN_2024.xlsx')

In [ ]:
final['days_with_metrics'].sum()